<a href="https://colab.research.google.com/github/TijsS/jupyterNotebook/blob/master/Copy_of_WijnOnderzoek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import seaborn as sns; sns.set()
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Hier laden we de 'redwine.csv' dataset op uit onze github. Vanwege de Portugese namen gebruiken we 'cp1252' encoding.

In [2]:
url = 'redwine.csv'
df = pd.read_csv(url,sep=';',index_col=0,encoding='cp1252')
df.head()

FileNotFoundError: ignored

Zoals hieronder te zien missen er wat kollomen zoals bijvoorbeeld 'alcohol' dit komt omdat ze niet goed zijn omgezet naar de float datatype

In [0]:
df.describe()
df.dtypes

Hier zetten we de datatype van de kolom om naar de float type zodat we er mee kunnen rekenen. De 'foutieve' waardes die in de kollom zitten maar niet kunnen worden omgezet naar float worden NaN waardes.

In [0]:
df['alcohol'] = df['alcohol'].apply(pd.to_numeric, errors='coerce', downcast='float')
df['density'] = df['density'].apply(pd.to_numeric, errors='coerce', downcast='float')
df['citric acid'] = df['citric acid'].apply(pd.to_numeric, errors='coerce', downcast='float')
df['points'] = df['points'].apply(pd.to_numeric, errors='coerce', downcast='float')
df.dtypes

hier wordt de missende data/foutieve data met de gemiddelde waardes van die kollom gevult.

In [0]:
df = df.fillna(df.mean())

Hieronder is te zien dat de kolomen die eerder niet te zien waren (bijv. alcohol) nu wel te zijn zijn en dat er dus mee te rekenen is.



In [0]:
df.describe()

#Data verkenning 
We gaan nu individueel naar specifieke kolommen kijken.

provinces:

In [0]:
provincies = df['province'].nunique()
print("we werken met " + str(provincies) + " verschillende gebieden waar de wijn vandaan komen")

Prijs:

In [0]:
gemiddeldePrice = df['price'].mean()
maxPrice = df['price'].max()

print("de gemiddelde prijs van de wijnen is: " + str(gemiddeldePrice) + " euro")
print("de maximale prijs van de wijnen is: " + str(maxPrice) + " euro")


Hieronder in de boxplot van de prijs van de wijn is te zien dat er erg veel uitschieters zijn. Dit zal ons dan ook kunnen helpen in de deelvragen 'Kun je op basis van de gemiddelde temperatuur wat zeggen over de prijs?' en 'Zorgt de prijs voor een wijn voor een significant verschil t.o.v. de score van de wijn?'.

In [0]:
boxplot = df[['price']].boxplot(grid=True, rot=90, fontsize=15)

Score:

In [0]:
gemiddeldePoints = df['points'].mean()
maxPoints = df['points'].max()
minPoints = df['points'].min()

print("Het minimale aantal punten van de wijnen is: " + str(minPoints)+ " punten")
print("de gemiddelde aantal punten van de wijnen is: " + str(gemiddeldePoints) + " punten")
print("de maximale aantal punten van de wijnen is: " + str(maxPoints) + " punten")

In de boxplot is te zien dat er 2 uitschieters zijn. Terugkijkend naar de boxplot over prijs is te zien dat er ook 2 extreme uitschieters waren. Als dit dezelfde wijnen zijn lijkt het op eerste gezicht dat er misschien inderdaad een verband kan zijn tussen prijs en score.

In [0]:
boxplot = df[['points']].boxplot(grid=True, rot=90, fontsize=15)

# Deelvraag: In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?


Bij deze vraag gaan wij gebruik maken van unsupervised machine learning om er achter te komen hoe de score van deze wijnen
samenhangt met de chemische samenstelling. Hiervoor zullen wij scikit learn gebruiken.

Als bonus willen wij mocht hier de tijd voor zijn bepaalde keywords voor verschildende chemische samenstellingen te vinden.
Dit willen wij doen door middel van de description.


In [0]:
bins =(80, 91, 101)
group_names = ['bad', 'good']
df_sml = df.drop(['country', 'description','designation','province','taster_name','title', 'variety', 'winery'], axis = 1)
df_sml['points'] = pd.cut(df['points'], bins = bins, labels = group_names)
df_sml['points'].unique()

In [0]:
label_points = LabelEncoder()

In [0]:
df_sml['points'] = label_points.fit_transform(df_sml['points'])
df_sml.head(50)

In [0]:
X = df_sml.drop('points',axis = 1)
y = df_sml['points']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 77 )

In [0]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [0]:
pred_rfc[:1200]

In [0]:
print(classification_report(y_test, pred_rfc))
print(confusion_matrix(y_test, pred_rfc))
cm = accuracy_score(y_test, pred_rfc)
cm

In [0]:
clf = svm.SVC()
clf.fit(X_train,y_train)
pred_clf = clf.predict(X_test)

In [0]:
print(classification_report(y_test, pred_clf))
print(confusion_matrix(y_test, pred_clf))
cm = accuracy_score(y_test, pred_clf)
cm

In [0]:
mlpc = MLPClassifier(hidden_layer_sizes=(7,7,7),max_iter=1500)
mlpc.fit(X_train,y_train)
pred_mlpc = mlpc.predict(X_test)

In [0]:
print(classification_report(y_test, pred_mlpc))
print(confusion_matrix(y_test, pred_mlpc))
cm = accuracy_score(y_test, pred_mlpc)
cm

# Deelvraag: Heeft de proever voorkeur voor een bepaalde pH-waarde en kunnen we hierdoor de gegeven punten van een wijn voorspellen?

Voor deze vraag pakken wij 3 proevers met de meeste reviews. De score van de wijn vergelijken met de ph waarde. Als we hierin een verband zien kunnen we door middel van supervised machine learning de score voorspellen.

We pakken de 3 mensen met de meeste reviews. Dit doen we omdat we anders mensen gaan krijgen met een te kleine sample size. 

In [0]:
dfD2 = df.copy()
dfD2.taster_name.value_counts()


Het blijkt dat de top 3 bestaat uit: Roger voss, Michael Schachner en Kerin O'Keefe.

In [0]:
#maak voor alle een eigen dataframe aan met alleen de nodige waardes
dfRoger = dfD2[['points', 'alcohol']].loc[df['taster_name'] == 'Roger Voss']
dfMichael = dfD2[['points', 'alcohol']].loc[df['taster_name'] == 'Michael Schachner']
dfKerin = dfD2[['points', 'alcohol']].loc[df['taster_name'] == 'Kerin O’Keefe']

In [0]:
#haal de uitschieters uit de dataframe
Q1 = dfRoger['alcohol'].quantile(0.25)
Q3 = dfRoger['alcohol'].quantile(0.75)
IQR = Q3 - Q1

filter = (dfRoger['alcohol'] >= Q1 - 1.5 * IQR) & (dfRoger['alcohol'] <= Q3 + 1.5 *IQR)
dfRoger = dfRoger.loc[filter] 

Q1 = dfRoger['points'].quantile(0.25)
Q3 = dfRoger['points'].quantile(0.75)
IQR = Q3 - Q1

filter = (dfRoger['points'] >= Q1 - 1.5 * IQR) & (dfRoger['points'] <= Q3 + 1.5 *IQR)
dfRoger = dfRoger.loc[filter] 

#normaliseer waardes
dfRoger=(dfRoger-dfRoger.mean())/dfRoger.std()

#plot een grafiek om te zien hoeveel clusters er zijn. In dit geval zijn dat er 4
ax = sns.jointplot('alcohol', 'points', data=dfRoger, kind="kde", color="g")

In [0]:
Q1 = dfMichael['alcohol'].quantile(0.25)
Q3 = dfMichael['alcohol'].quantile(0.75)
IQR = Q3 - Q1

filter = (dfMichael['alcohol'] >= Q1 - 1.5 * IQR) & (dfMichael['alcohol'] <= Q3 + 1.5 *IQR)
dfMichael = dfMichael.loc[filter] 

Q1 = dfMichael['points'].quantile(0.25)
Q3 = dfMichael['points'].quantile(0.75)
IQR = Q3 - Q1

filter = (dfMichael['points'] >= Q1 - 1.5 * IQR) & (dfMichael['points'] <= Q3 + 1.5 *IQR)
dfMichael = dfMichael.loc[filter] 

#normaliseer waardes van Michael 
dfMichael=(dfMichael-dfMichael.mean())/dfMichael.std()
#plot een grafiek om te zien hoeveel clusters er zijn. In dit geval zijn dat er 3
ax = sns.jointplot('alcohol', 'points', data=dfMichael, kind="kde", color="g")

In [0]:
Q1 = dfKerin['alcohol'].quantile(0.25)
Q3 = dfKerin['alcohol'].quantile(0.75)
IQR = Q3 - Q1

filter = (dfKerin['alcohol'] >= Q1 - 1.5 * IQR) & (dfKerin['alcohol'] <= Q3 + 1.5 *IQR)
dfKerin = dfKerin.loc[filter] 

Q1 = dfKerin['points'].quantile(0.25)
Q3 = dfKerin['points'].quantile(0.75)
IQR = Q3 - Q1

filter = (dfKerin['points'] >= Q1 - 1.5 * IQR) & (dfKerin['points'] <= Q3 + 1.5 *IQR)
dfKerin = dfKerin.loc[filter] 

#normaliseer waardes van Kerin 
dfKerin=(dfKerin-dfKerin.mean())/dfKerin.std()

#plot een grafiek om te zien hoeveel clusters er zijn. In dit geval zijn dat er 2
ax = sns.jointplot('alcohol', 'points', data=dfKerin, kind="kde", color="g")

Nu we weten hoeveel clusters ieder persoon heeft gaan we ze met gmm/kmeans plotten.

Hier word gebruik gemaakt van GMM door de oneven aantal clusters. Ook doordat de clusters meer een eclipse achtige vorm hebben. Hierbij kun je beter GMM gebruiken.

In [0]:
import numpy as np
from sklearn import mixture
from sklearn.mixture import GaussianMixture as GMM

mixture.GaussianMixture(dfRoger)

gmm = GMM(n_components=4, n_init = 450, covariance_type='full').fit(dfRoger)
label= True
ax = None
ax = ax or plt.gca()
labels = gmm.fit(dfRoger).predict(dfRoger)

ax.scatter(dfRoger['alcohol'], dfRoger['points'], c=labels, s=40, cmap='viridis', zorder=2)

De groep rond de [-1,1] kun je de hoogscorende wijnen noemen met een wat lager alcohol gehalte.
de groep rond [-1,-1] is meer het overal gemmidelde, hier zijn de meeste wijnen het is een beetje de matige score wijnig alcohol groep.
dan komt de groep bij [0,-2] Dit zijn een beetje de slechte wijnen, de alcohol gehalte heeft niet niet veel invloed.
De laatste groep is bij [2,1] Dit zijn de wijnen met de beste score en ook het meeste alcohol. 

Hier word gebruik gemaakt van GMM door de oneven aantal clusters. Ook doordat de clusters meer een eclipse achtige vorm hebben. Hierbij kun je beter GMM gebruiken.

In [0]:
mixture.GaussianMixture(dfMichael)

gmm = GMM(n_components=3, n_init = 300, covariance_type='full').fit(dfMichael)
label= True
ax = None
ax = ax or plt.gca()
labels = gmm.fit(dfMichael).predict(dfMichael)

ax.scatter(dfMichael['alcohol'], dfMichael['points'], c=labels, s=40, cmap='viridis', zorder=2)

De groep rond de [0,1] kun je de hoogscorende wijnen noemen. de groep rond [-1,-1] is meer het overal gemmidelde, hier zijn de meeste wijnen het is een beetje de matige score wijnig alcohol groep. dan komt de groep bij [0,-2] Dit zijn een beetje de slechte wijnen, de alcohol gehalte heeft niet niet veel invloed.

Hier is gebruik gemaakt van k-means omdat dit een iets eenvoudigere methode is en het hier kan doordat het een even aantal clusters is.

In [0]:
from sklearn.cluster import KMeans

# doe de clustering
kerMeans = KMeans(n_clusters=2).fit(dfKerin)

# druk de middelpunten af
print(kerMeans.cluster_centers_)

dfKerin['Groep'] = kerMeans.labels_

plt.scatter(dfKerin['alcohol'], dfKerin['points'], c=dfKerin['Groep'], cmap='plasma')
plt.xlim(-2,3)
plt.ylim(-3,3)

plt.show()

De 2 groepen zou je kunnen beschrijven als de "wijnig alcohol matige score, wijnen" en de "hogere score meer alcohol, wijnen"

# Deelvraag: Zorgt de prijs voor een wijn voor een significant verschil t.o.v. de score van de wijn?

Wij gaan hier onderzoekdoen naar de relatie tussen de prijs en de score van de wijn. Dit doen wij doormiddel van een
scatterplot met trend lijn. Ook zullen wij de wijnen opdelen in prijscatagorien en hier vervolgens de gemidelde prijs voor berekennen.

In [0]:
dfGoedkoop = df[df['price'] <= 5]
dfMedium = df[(df['price'] > 5) & (df['price'] <= 13)]
dfDuur = df[(df['price'] > 13) & (df['price'] <= 50)]
dfExclusief = df[df['price'] > 50]

objects = ('Goodkope Wijn\n(€5 of minder)', 'Medium\n(€5 tot €13)', 'Duur\n(€13 tot €50)', 'Exclusief\n(Meer dan €50)')
y_pos = np.arange(len(objects))
catag = [dfGoedkoop['points'].mean(),dfMedium['points'].mean(), dfDuur['points'].mean(),dfExclusief['points'].mean()]

plt.bar(y_pos, catag, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
# for index, value in enumerate(catag):
#     plt.text(value, index, str(value))
plt.ylabel('Points')
plt.title('Gemidelde score per prijscatagorie')

plt.show()

In [0]:
ax = sns.jointplot('points', 'price', data=df,kind="reg")

# Deelvraag: Kun je op basis van de gemiddelde temperatuur wat zeggen over de prijs?

De dataset van de temperaturen per (wijn)gebied wordt ingeladen. Weer met gebruik van 'cp1252' encoding.

In [0]:
urlTemperatureDataset = 'dataset.csv'
Tempdf = pd.read_csv(urlTemperatureDataset,sep=';',encoding='cp1252')
#copy van de nodige data voor deelvraag 4
dfD4 = df[['province','price']].copy()
#sommige wijnen hebben 'foutieve' provincies zoals bijvoorbeeld 'portugal' 
#dit is natuurlijk niet bruikbaar. Deze waardes halen we hieronder weg
dfD4 = dfD4[~dfD4['province'].isin(['Portugal', 'Portuguese Table Wine',"Vinho da Mesa","Table wine", "Vinho Licoroso","Vinho Verde"]) ]
#voor voorbeeld van de temperature dataset
Tempdf.head()


Eerst berekenen we de gemiddelde prijs per gebied. Hierna plaatsen we de gemmidelde temperatuur over een map van portugal in kleuren. We zetten dan de als we dit hebben gedaan de gemmidelde prijzen op de juiste locatie op de map van portugal. Nadat je dit hebt gedaan heb je een mooi overzicht van de prijs/temperatuur correlatie. Hiermee kun je dan de deelvraag beantwoorden.

Veel van de genoemde gebieden in de wijndataset zijn veroudert en hebben nu een nieuwe naam. Of gebruiken ze verschillende wijn gebied labels (DOC, VR). Om deze reden zijn sommige gebieden samen gegroepeerd omdat het eigenlijk dus (ongeveer) hetzelfde gebied is.

In [0]:
#groepeer Alentejano en Alentejo door de provincie Alentejano te veranderen naar Alentejo
dfD4.loc[dfD4.province == 'Alentejano', 'province'] = "Alentejo"
#Hier doen we hetzelfde voor andere provincies
dfD4.loc[dfD4.province == 'Dão', 'province'] = "Terras do Dão"
dfD4.loc[dfD4.province == 'Bairrada', 'province'] = "Beira Atlantico"
dfD4.loc[dfD4.province == 'Douro', 'province'] = "Duriense"
dfD4.loc[dfD4.province == 'Estremadura', 'province'] = "Lisboa"
dfD4.loc[dfD4.province == 'Ribatejano', 'province'] = "Tejo"
dfD4.loc[dfD4.province == 'Ribatejo', 'province'] = "Tejo"
dfD4.loc[dfD4.province == 'tejo', 'province'] = "Tejo"
dfD4.loc[dfD4.province == 'Península de Setúbal', 'province'] = "Setubal"
dfD4.loc[dfD4.province == 'Terras do Sado', 'province'] = "Setubal"
dfD4.loc[dfD4.province == 'Algarve', 'province'] = "faro"

In [0]:
#hier maken we de column temperature aan bij de df, hierin stoppen we de temperatuur van de externe dataframe per regio
dfD4['temperature'] = 0
dfD4.loc[dfD4.province == 'Alenquer', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Alenquer"].values[0]
dfD4.loc[dfD4.province == 'Alentejo', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Alentejano/Alentejo"].values[0]
dfD4.loc[dfD4.province == 'Beira Atlantico', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Bairrada/Beira Atlantico"].values[0]
dfD4.loc[dfD4.province == 'Beira Interior', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Beira Interior"].values[0]
dfD4.loc[dfD4.province == 'Beiras', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Beiras"].values[0]
dfD4.loc[dfD4.province == 'Duriense', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Douro/Duriense"].values[0]
dfD4.loc[dfD4.province == 'Lisboa', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Estremadura/Lisboa"].values[0]
dfD4.loc[dfD4.province == 'Minho', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Minho"].values[0]
dfD4.loc[dfD4.province == 'Obidos', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Obidos"].values[0]
dfD4.loc[dfD4.province == 'Palmela', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "palmela "].values[0]
dfD4.loc[dfD4.province == 'Setubal', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Península de Setúbal/Setubal"].values[0]
dfD4.loc[dfD4.province == 'Tejo', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Ribatejano/Ribatejo/tejo"].values[0]
dfD4.loc[dfD4.province == 'Terras do Dão', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Dão/Terras do Dão"].values[0]
dfD4.loc[dfD4.province == 'Trás-os-Montes', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Trás-os-Montes"].values[0]
dfD4.loc[dfD4.province == 'faro', 'temperature'] = Tempdf[['temp']].loc[Tempdf.province == "Algarve/faro"].values[0]


In [0]:
dfD4.dtypes

De datatype van temperature is nu object we maken dit een float waarde.

In [0]:
dfD4['temperature'] = dfD4['temperature'].astype(float)
dfD4.dtypes

In [0]:
#hier is de gemiddelde prijs per temperatuur berekend en afgerond op 2 decimalen.
dfD4TempPrice = dfD4.groupby(['temperature']).mean().round(2)
dfD4TempPrice = dfD4TempPrice.reset_index()
dfD4TempPrice

in de scatterplot en de grafiek hierboven kun je misschien een heele lichte correlatie zien tussen temperatuur en price, hoe warmer het weer hoe goedkoper de wijn. dit is natuurlijk ook wel logisch zijn. Een wijnboer met warm weer kan meer druiven krijgen, De qualiteit van deze druiven zullen alleen wat minder zijn.

In [0]:
ax = sns.jointplot(x="temperature", y="price", data=dfD4TempPrice)

In [0]:
#Haal bij beide kolommen per waarde de gemmidelde van de kolom af
xTemp = dfD4TempPrice['temperature'] - dfD4TempPrice['temperature'].mean()
yPrice = dfD4TempPrice['price'] - dfD4TempPrice['price'].mean()


#Voor het bijhouden welke row we zijn
c=0

#We gaan langs elke row en hier de xTemp vermenigvuldigen met de yPrice, Hier nemen we dan de som van.
somXY = 0 

for i in xTemp:
  somXY += xTemp[c] * yPrice[c]
  c +=1

#hier berekenen we de correlatieCoefficent door somXY te pakken en 
#dit delen door het aantal waardes keer de standaarddeviaties van de kolommen
print("de correlatiecoefficent is: " + str(somXY/(12*dfD4TempPrice['price'].std() * dfD4TempPrice['temperature'].std())))

In [0]:
#We controleren nog even of de correlatie correct is berekent.
dfD4TempPrice.corr()

Zo te zien was het goed berekent. Doordat de waarde te ver ligt van tussen de 1.0 en 0,8 en de -1.0 en -0.8 kunnen we definitief zeggen dat er geen correlatie is tussen prijs en temperatuur. We kunnen dus niet op basis van de gemiddelde temperatuur wat zeggen over de prijs.

Hieronder visualiseren we nog even de gegevens per province.

In [0]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

print("Kies hier een provintie waarvan u de boxplot wilt zien.")
#Maak een dropdown menu met hierin alle provinces.
interact(f, x=[('Minho',1), ('Alentejano/Alentejo',2), ('Trás-os-Montes',3), ('Beira Interior',4), ('Dão/Terras do Dão', 5), ('Bairrada/Beira Atlantico',6), ('Douro/Duriense', 7), ('Obidos',8), ('Beiras',9), ('Estremadura/Lisboa', 10), ('Alenquer',11), ('palmela',12), ('Ribatejano/Ribatejo/tejo',13), ('Península de Setúbal/Setubal', 14), ('Algarve/faro',15)], description='Province:');

#functie die aangeroepen wordt als je een province selecteert. Hier zetten we de naam van de geselecteerde province in 'name'
def f(x):
    global name
    if x == 1:
      name = "Minho"
    if x == 2:
      name = "Alentejo"
    if x == 3:
      name = 'Trás-os-Montes'
    if x == 4:
      name = 'Beira Interior'
    if x == 5:
      name = 'Terras do Dão'
    if x == 6:
      name = 'Beira Atlantico'
    if x == 7:
      name = 'Duriense'
    if x == 8:
      name = 'Obidos'
    if x == 9:
      name = 'Beiras'
    if x == 10:
      name = 'Lisboa'
    if x == 11:
      name = 'Alenquer'
    if x == 12:
      name = 'Palmela'
    if x == 13:
      name = 'Tejo'
    if x == 14:
      name = 'Setubal'
    if x == 15:
      name = 'faro'


In [0]:
#voer dit uit om de door u gekozen boxplot te krijgen.
import holoviews as hv
from holoviews import opts
hv.extension('plotly')
#maak de boxplot met de gekozen naam
print(name)
box = hv.BoxWhisker(data=dfD4.loc[dfD4.province == name], vdims='price')
box.opts(height=400, width=600)

# Deelvraag: Hebben de wijnen van afgelopen jaar een significant verschillende zuurgraad dan gemiddeld?

In het afgelopen jaar was het veel zonniger in Portugal dan jaren ervoor. Wijn verbouwers vragen zich af of de hoeveelheid zon het pH beinvloed. De dataset is de populatie met een betrouwbaarheid van 95%.

In [3]:
dfpH = df.copy()
dfpH.pH.value_count()

NameError: ignored